<a href="https://colab.research.google.com/github/SampMark/ETL-de-dados-da-PNP/blob/main/GitHub_ETL_of_PNP_Data___Pipeline_for_BigQuery___public_servants_dimension.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Extração da Plataforma Nilo Peçanha (PNP), filtragem, tratamento e armazenamento em Big Query dos dados de Instituições Federais de Ensino (IFES) - tratamento para dimensão `servidores`**

Este notebook automatiza a extração, tratamento e carga (ETL) dos microdados da Plataforma Nilo Peçanha (PNP) para o Google BigQuery. O fluxo foi projetado em etapas sequenciais para garantir a integridade e a qualidade dos dados.

## **Fluxo de Execução**

1. **Etapa 1: Instalação e Autenticação**: instalação das bibliotecas Python necessárias para a manipulação dos dados e conexão com os serviços Google (`gspread`, `pandas-gbq`, etc.). Autenticação do usuário e montagem do Google Drive para acesso e armazenamento dos arquivos de dados.
2. **Etapa 2: Definição das Funções Principais**: carregamento das funções em memória que realizam as principais tarefas do pipeline: download, descompressão, análise de cabeçalhos, processamento e tratamento dos dados.
3. **Etapa 3: Configuração do Pipeline**: o usuário define os parâmetros da extração através de uma interface interativa:

    * **Período**: define o intervalo de anos (Ano Inicial e Final) para a extração.
    * **Instituição(ões)**: filtra os dados para uma ou mais instituições específicas.
    * **Força atualização**: opção para baixar novamente os arquivos da PNP, ignorando o cache local no Google Drive.

4. **Etapa 4: Download e Análise de Cabeçalhos**: o script baixa os arquivos de dados compactados (`.gz`) da PNP para o Google Drive com base nas configurações da etapa anterior. Em seguida, os arquivos são descompactados e o script realiza uma análise comparativa dos cabeçalhos (colunas) de cada ano, exibindo uma tabela que destaca as diferenças.
    * **Tabela com o comparativo de cabeçalhos dos CSV processados**: permite a analise prévia das colunas a serem extraídas do conjunto de dados (ex: matriculas, servidores).
    * **Ponto de Decisão**: é gerada uma lista com as colunas comuns a todos os arquivos do período, que servirá de sugestão para a próxima etapa.

5. **Etapa 5: Seleção de Colunas e Processamento dos Dados**: esta etapa demanda uma **Ação do Usuário** que deve copiar a lista de colunas sugerida e editá-la conforme a necessidade, em seguida inserir na célula de código desta etapa. O script então processa os arquivos CSV, unificando-os em um único DataFrame (`df_filtrado`), mantendo apenas as colunas selecionadas e aplicando o filtro de instituição.

6. **Etapa 6: Tratamento e Limpeza dos Dados** é uma fase crucial do pipeline em que o DataFrame bruto (`df_filtrado`), que contém os dados extraídos de múltiplos anos, é transformado em um conjunto de dados "limpo", padronizado e enriquecido (`df_tratado`), pronto para análise, conforme implementação a seguir:
    *  **Renomeação de Colunas:** a maioria da colunas foirenomeadas para um padrão consistente (removendo espaços e caracteres especiais) para garantir a compatibilidade com o BigQuery e facilitar o acesso no código. Por exemplo, `Jornada de Trabalho` foi alterado para `Jornada_de_Trabalho`.
    *  **Padronização de Valores:** foram corrigidas inconsistências em colunas categóricas. Por exemplo, na coluna `Vinculo_Contrato`, o valor `Substituto/Temporario` foi corrigido para `Substituto/Temporário` , e na coluna `Titulacao`, `Aperfeicoamento` foi ajustado para `Aperfeiçoamento`.
    * **Limpeza dos Nomes dos Campi:** Os nomes na coluna `Campus_do_IFRN` foram limpos, removendo prefixos como "Campus " e "Campus Avançado " para simplificar a visualização (ex: "Campus Natal Central" tornou-se "Natal Central").
    *  **Criação de uma Nova Coluna (`Nivel_Educacional`):** Uma nova coluna chamada `Nivel_Educacional` foi criada a partir da coluna `Titulacao`, apenas para agrupar as diversas titulações em categorias hierárquicas mais amplas, como `(0) Educação Básica`, `(2) Educação Superior` e `(3) Pós-graduação`, facilitando análises agregadas sobre o nível de formação dos servidores.
7. **Etapa 7: Análise e Exportação para o BigQuery**: são realizadas análises estatísticas descritivas e de _outliers_ sobre os dados tratados para verificar a qualidade final. Finalmente, o DataFrame (`df_tratado`) é exportado para uma tabela no Google BigQuery, finalizando o processo de ETL. O script gera um link direto para a tabela criada.

<img src="https://www2.ifal.edu.br/noticias/ifal-se-destaca-na-eficiencia-academica-dos-institutos-federais-do-nordeste/plataforma-nilo-pecanha/@@images/98c1a2a4-6c59-436f-bdce-effa7ae4d539.jpeg" alt="Logo da Plataforma Nilo Peçanha" width="250"/>

In [ ]:
# -*- coding: utf-8 -*-
"""
Extração de microdados da PNP para Big Query

Este notebook automatiza o fluxo de trabalho com os microdados extraídos da Plataforma Nilo Peçanha (PNP),
permitindo a extração de diferentes tabelas e a análise de seus cabeçalhos em cada anos.
"""

# @title **ETAPA 1: Instalação de dependências, importações e autenticação do usuário no Google Drive**

# Instalação de Dependências
!pip install gspread gspread-dataframe oauth2client pandas-gbq --quiet
print("Dependências instaladas com sucesso!")

In [ ]:
# Importação de bibliotecas
import re
import pandas as pd
import numpy as np
import requests
import gspread
import gzip
import shutil
import ipywidgets as widgets
from IPython.display import display, HTML
from google.colab import auth, drive
from google.auth import default
from pathlib import Path
from typing import List, Dict, Set

In [ ]:
# Autenticação e Montagem do Google Drive
try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
    drive.mount('/content/drive')
    print("\nAutenticação e montagem do Google Drive realizadas com sucesso!")
except Exception as e:
    print(f"Ocorreu um erro durante a autenticação ou montagem do Drive: {e}")

In [ ]:
# @title **ETAPA 2: Definição das Funções Principais**

def download_pnp_data(table_name: str, start_year: int, end_year: int, force_update: bool):
    """
    Baixa os arquivos da PNP para uma pasta específica no Google Drive.
    O nome do arquivo e a pasta de destino são baseados no table_name.
    """
    drive_folder = Path(f'/content/drive/MyDrive/Coisas do IFRN/Prodes/Indicadores/PNP/{table_name.capitalize()}')
    drive_folder.mkdir(parents=True, exist_ok=True)
    print(f"Verificando arquivos na pasta do Google Drive: {drive_folder}")

    base_url = "https://d236w85zd3t8iw.cloudfront.net/pnp-tests/microdados"

    for year in range(start_year, end_year + 1):
        file_name = f"microdados_{table_name}_{year}.csv.gz"
        url = f"{base_url}/{year}/{file_name}"
        destination = drive_folder / file_name

        if not force_update and destination.exists():
            print(f"✔ O arquivo para {year} ('{file_name}') já existe. Usando o cache do Drive.")
            continue

        try:
            print(f"⬇ Baixando dados para {year} de {url}...")
            with requests.get(url, stream=True) as r:
                r.raise_for_status()
                with open(destination, 'wb') as f:
                    shutil.copyfileobj(r.raw, f)
            print(f"✔ Download de {year} concluído com sucesso.")
        except requests.exceptions.RequestException as e:
            print(f"❌ Falha ao baixar o arquivo para {year}. Erro: {e}. O arquivo pode não existir para este ano.")

def decompress_gz_to_csv(gz_path: Path, out_dir: Path) -> Path:
    """Descompacta cada arquivo .gz para a pasta de trabalho"""
    out_dir.mkdir(parents=True, exist_ok=True)
    # Remove a extensão .gz para obter o nome do arquivo CSV
    csv_out_path = out_dir / gz_path.with_suffix("").name
    print(f"Descompactando: {gz_path.name} -> {csv_out_path.name}")
    with gzip.open(gz_path, "rb") as f_in, open(csv_out_path, "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)
    return csv_out_path

def get_header(path: Path, sep: str = ';') -> List[str]:
    """Lê o cabeçalho de cada arquivo CSV descompactado"""
    try:
        return list(pd.read_csv(path, nrows=0, sep=sep, engine='python').columns)
    except Exception as e:
        print(f"Erro ao ler o cabeçalho de {path.name}: {e}")
        return []

def analyze_and_compare_headers(csv_paths: List[Path]) -> pd.DataFrame:
    """
    Cria um DataFrame comparativo de cabeçalhos e sugere colunas comuns.
    """
    if not csv_paths:
        print("Nenhum arquivo CSV para analisar.")
        return pd.DataFrame(), []

    headers_dict = {path.name: get_header(path) for path in csv_paths}

    # Identificar colunas comuns
    sets_of_headers = [set(h) for h in headers_dict.values() if h]
    if not sets_of_headers:
        print("Não foi possível ler nenhum cabeçalho.")
        return pd.DataFrame(), []

    common_columns = sorted(list(sets_of_headers[0].intersection(*sets_of_headers[1:])))

    # Criar DataFrame para comparação visual
    all_columns = sorted(list(set.union(*sets_of_headers)))
    comparison_data = {}
    for col in all_columns:
        comparison_data[col] = [("✔" if col in headers_dict.get(fname, []) else "❌") for fname in headers_dict.keys()]

    comparison_df = pd.DataFrame(comparison_data, index=headers_dict.keys()).transpose()

    print("\n--- Análise de Cabeçalhos Concluída ---")
    print("A tabela abaixo mostra quais colunas estão presentes (✔) ou ausentes (❌) em cada arquivo.")
    display(HTML(comparison_df.to_html()))

    print("\n--- Sugestão de Colunas Comuns ---")
    print(f"Foram encontradas {len(common_columns)} colunas presentes em TODOS os arquivos do período:")
    # Imprime a lista formatada para ser copiada e colada
    print("\nmanter_colunas = [")
    for col in common_columns:
        print(f"    '{col}',")
    print("]")

    return comparison_df, common_columns

def process_to_dataframe(csv_paths: List[Path], columns_to_keep: List[str], institutions: List[str], chunksize: int = 100000, sep: str = ';'):
    """
    Unifica, filtra e concatena os CSVs em um único DataFrame,
    mantendo apenas a lista de colunas fornecida.
    """
    if not csv_paths:
        raise RuntimeError("Nenhum arquivo CSV para processar.")
    if not columns_to_keep:
        raise ValueError("A lista 'columns_to_keep' não pode estar vazia.")

    print(f"\nProcessamento iniciado. Serão importadas {len(columns_to_keep)} colunas pré-definidas.")

    # Encontrar coluna da instituição (considerando inconsistências de codificação)
    col_inst = None
    if 'Instituição' in columns_to_keep:
        col_inst = 'Instituição'
    elif 'InstituiÃ§Ã£o' in columns_to_keep:
        col_inst = 'InstituiÃ§Ã£o'

    if institutions and col_inst:
        print(f"Filtrando pela coluna '{col_inst}' com os valores: {institutions}")
    elif institutions:
        print("AVISO: Filtro de instituição solicitado, mas a coluna 'Instituição' não está na lista de colunas a serem mantidas.")

    institution_map = {'Instituto Federal do Rio Grande do Norte': 'IFRN'}
    df_list = []

    for csv_path in csv_paths:
        print(f"Processando e filtrando: {csv_path.name}")
        try:
            # Lê o cabeçalho do arquivo para saber quais colunas ele realmente tem
            actual_header = get_header(csv_path, sep)
            # Usa apenas as colunas da nossa lista que existem neste arquivo
            cols_to_read = [col for col in columns_to_keep if col in actual_header]

            for chunk in pd.read_csv(csv_path, usecols=cols_to_read, chunksize=chunksize, sep=sep, engine='python', on_bad_lines='warn'):
                if col_inst and col_inst in chunk.columns:
                    chunk[col_inst] = chunk[col_inst].replace(institution_map)
                    if institutions:
                        chunk = chunk[chunk[col_inst].isin(institutions)]

                if not chunk.empty:
                    df_list.append(chunk)
        except Exception as e:
            print(f"  ERRO ao processar o arquivo {csv_path.name}: {e}. Pulando este arquivo.")
            continue

    if not df_list:
        print("AVISO: Nenhum dado encontrado para as instituições selecionadas ou os arquivos estavam vazios.")
        return pd.DataFrame(columns=columns_to_keep)

    final_df = pd.concat(df_list, ignore_index=True)
    # Garante que o DataFrame final tenha todas as colunas da lista, preenchendo com NaN as que não existiam
    final_df = final_df.reindex(columns=columns_to_keep)

    print(f"\nProcesso concluído. DataFrame final criado com {len(final_df):,} linhas e {len(final_df.columns)} colunas.")
    return final_df

In [ ]:
# @title **ETAPA 3: Configuração do Processo e Download (Opção para Código da Unidade)**

# --- Interface Interativa de Configuração ---
style = {'description_width': 'initial'}

# Seleção da tabela
table_name_dropdown = widgets.Dropdown(
    options=['matriculas', 'eficiencia_academica', 'financeiro', 'servidores'],
    value='matriculas',
    description='Tabela de Dados:',
    style=style
)

# Período de anos
start_year_slider = widgets.IntSlider(value=2017, min=2017, max=2024, step=1, description='Ano Inicial:', style=style)
end_year_slider = widgets.IntSlider(value=2024, min=2017, max=2024, step=1, description='Ano Final:', style=style)

# Opção de forçar atualização
force_update_checkbox = widgets.Checkbox(value=False, description='Forçar atualização (baixar novamente os arquivos existentes)', style=style)

# --- Filtro de instituições por código ---
# O campo pede o código numérico da instituição.
institution_code_text = widgets.Text(
    value='26435', # O valor padrão está preenchido com o código do IFRN = '26435'
    description='Códigos das Instituições (Co Inst):',
    style=style,
    layout=widgets.Layout(width='50%')
)

print("--- Configure os Parâmetros do Pipeline ---")
display(table_name_dropdown)
display(widgets.HBox([start_year_slider, end_year_slider]))
display(force_update_checkbox)
display(institution_code_text)

In [ ]:
# @title **ETAPA 3: Configuração do Processo e Download**

# --- Interface Interativa de Configuração ---
style = {'description_width': 'initial'}

# Seleção da tabela
table_name_dropdown = widgets.Dropdown(
    options=['matriculas', 'eficiencia_academica', 'financeiro', 'servidores'],
    value='servidores',
    description='Tabela de Dados:',
    style=style
)

# Período de anos
start_year_slider = widgets.IntSlider(value=2017, min=2017, max=2024, step=1, description='Ano Inicial:', style=style)
end_year_slider = widgets.IntSlider(value=2024, min=2017, max=2024, step=1, description='Ano Final:', style=style)

# Opção de forçar atualização
force_update_checkbox = widgets.Checkbox(value=True, description='Forçar atualização (baixar novamente os arquivos existentes)', style=style)

# --- Filtro de instituições por nome ---
# O campo agora aceita um ou mais nomes de instituições, separados por vírgula.
# O valor padrão já inclui as duas variações para o IFRN.
institution_name_text = widgets.Text(
    value='IFRN, Instituto Federal do Rio Grande do Norte',
    description='Nome da Instituição (use vírgula para múltiplos nomes):',
    style=style,
    layout=widgets.Layout(width='70%') # Largura aumentada para melhor visualização
)

print("--- Configure os Parâmetros do Pipeline ---")
display(table_name_dropdown)
display(widgets.HBox([start_year_slider, end_year_slider]))
display(force_update_checkbox)
display(institution_name_text)

In [ ]:
# @title **ETAPA 4: Download dos arquivos `.gz` e análise dos cabeçalhos dos `.csv` extraídos**

# 1. Pega os valores dos widgets da Etapa 3
table_name = table_name_dropdown.value
start_year = start_year_slider.value
end_year = end_year_slider.value
force_update = force_update_checkbox.value

# Aplica o nome correto do widget e analisa as strings separadas por vírgulas
institutions_str = institution_name_text.value
institutions_list = [inst.strip() for inst in institutions_str.split(',') if inst.strip()]

# 2. Executa o download
download_pnp_data(table_name, start_year, end_year, force_update)

# 3. Prepara os arquivos para a análise
drive_folder = Path(f'/content/drive/MyDrive/Coisas do IFRN/Prodes/Indicadores/PNP/{table_name.capitalize()}')
work_dir = Path.cwd() / "extracted_csvs"
if work_dir.exists(): shutil.rmtree(work_dir)
work_dir.mkdir()

input_files_gz = [drive_folder / f"microdados_{table_name}_{year}.csv.gz" for year in range(start_year, end_year + 1)]
input_files_gz_existing = [f for f in input_files_gz if f.exists()]

all_csvs = []
if input_files_gz_existing:
    for gz_file in input_files_gz_existing:
        all_csvs.append(decompress_gz_to_csv(gz_file, work_dir))
else:
    print("Nenhum arquivo .gz encontrado no Drive para o período e tabela selecionados.")

# 4. Analisa e compara os cabeçalhos
if all_csvs:
    comparison_df, common_columns = analyze_and_compare_headers(all_csvs)
else:
    print("Nenhuma análise de cabeçalho pôde ser feita, pois nenhum arquivo CSV foi descompactado.")

print("\n\n>>> AÇÃO NECESSÁRIA <<<")
print("Copie a lista de colunas comuns sugerida acima (ou edite-a conforme sua necessidade) e cole na célula da 'ETAPA 5' antes de executá-la.")

In [ ]:
# @title **ETAPA 5: Definição das colunas necessárias para extração de `df_filtrado`**

# >>> LISTA DE COLUNAS PARA EXTRAÇÃO (COLE AQUI) <<<
# Exemplo baseado na sugestão da etapa anterior.
# Edite esta lista conforme a necessidade.
# Lista de colunas a serem mantidas no DataFrame final, organizadas por categoria e finalidade.

manter_colunas = [
    'Classe',
    'Cod Unidade',
    'Código Municipio com DV',
    'Código da Unidade de Ensino - SISTEC',
    'Instituição',
    'Jornada de Trabalho',
    'Município',
    'Número de registros',
    'RSC',
    'Região',
    'Titulação',
    'Unidade de Lotação',
    'Vinculo Carreira',
    'Vinculo Contrato',
    'Vinculo Professor',
]

# --- Execução do Processamento ---
df_filtrado = None # Inicializa a variável
if not manter_colunas:
    print("❌ ERRO: A lista 'manter_colunas' está vazia. Preencha-a com as colunas desejadas e execute novamente.")
elif not all_csvs:
    print("❌ ERRO: Nenhum arquivo CSV foi encontrado para processar. Execute a Etapa 4 primeiro.")
else:
    # Cria o DataFrame filtrado com base na seleção de colunas
    df_filtrado = process_to_dataframe(
        csv_paths=all_csvs,
        columns_to_keep=manter_colunas,
        institutions=institutions_list
    )
    display(df_filtrado.head())

# --- Análise do DataFrame Gerado ---
if df_filtrado is not None:
    print("\n--- Análise Detalhada do DataFrame Final ---")
    if df_filtrado.empty:
        print("O DataFrame foi criado, mas está vazio (não contém linhas).")
    else:
        num_rows, num_cols = df_filtrado.shape
        print(f"Dimensões: {num_rows:,} linhas e {num_cols} colunas.")
        print("\nEstrutura e Tipos de Dados:")
        df_filtrado.info()
else:
    print("\nO DataFrame 'df_filtrado' não foi criado. Verifique os erros nas etapas anteriores.")

In [ ]:
# @title **Listagem e análise de valores únicos para as colunas indicadas em `df_filtrado`**
if df_filtrado is not None:
    columns_to_list = ['Classe', 'Jornada de Trabalho', 'Unidade de Lotação', 'RSC', 'Titulação', 'Vinculo Carreira', 'Vinculo Contrato',
                       'Vinculo Professor']

    for col in columns_to_list:
        if col in df_filtrado.columns:
            # Converte a coluna para o tipo 'string' antes de classificar valores exclusivos
            unique_values = sorted(df_filtrado[col].astype(str).unique())
            print(f"Valores únicos em ordem alfabética na coluna '{col}':")
            for value in unique_values:
                print(f"- {value}")
            print("-" * 20) # Separador
        else:
            print(f"A coluna '{col}' não foi encontrada no DataFrame 'df_filtrado'.")
else:
    print("DataFrame 'df_filtrado' não encontrado. Execute as etapas anteriores.")

In [ ]:
# @title **ETAPA 6: Tratamento e Limpeza dos Dados em `df_tratado`**

# Cria uma cópia de df_filtrado para os tratamentos necessários
if df_filtrado is not None:
    df_tratado = df_filtrado.copy()
    print("DataFrame 'df_tratado' criado como cópia de 'df_filtrado'.")

    # Renomeia colunas adequadamente para o BigQuery
    df_tratado = df_tratado.rename(columns={
        'Cod Unidade': 'Cod_Unidade',
        'Código Municipio com DV': 'Codigo_Municipio_com_DV',
        'Código da Unidade de Ensino - SISTEC': 'Codigo_da_Unidade_de_Ensino_-_SISTEC',
        'Instituição': 'Instituicao',
        'Jornada de Trabalho': 'Jornada_de_Trabalho',
        'Município': 'Municipio',
        'Número de registros': 'Numero_de_registros',
        'Região': 'Regiao',
        'Titulação': 'Titulacao',
        'Unidade de Lotação': 'Campus_do_IFRN',
        'Vinculo Carreira': 'Vinculo_Carreira',
        'Vinculo Contrato': 'Vinculo_Contrato',
        'Vinculo Professor': 'Vinculo_Professor',
    })

    # Ajuste do parâmetro na coluna 'Vinculo_Contrato'
    if 'Vinculo_Contrato' in df_tratado.columns:
        df_tratado['Vinculo_Contrato'] = df_tratado['Vinculo_Contrato'].replace(
            'Substituto/Temporario', 'Substituto/Temporário'
        )
        print("Parâmetro 'Substituto/Temporario' corrigido para 'Substituto/Temporário' na coluna 'Vinculo_Contrato'.")
    else:
        print("Aviso: Coluna 'Vinculo_Contrato' não encontrada em 'df_tratado'.")

    # Ajuste de parâmetro na coluna 'Titulacao'
    if 'Titulacao' in df_tratado.columns:
        df_tratado['Titulacao'] = df_tratado['Titulacao'].replace(
            'Aperfeicoamento', 'Aperfeiçoamento'
        )
        print("Parâmetro 'Aperfeicoamento' corrigido para 'Aperfeiçoamento' na coluna 'Titulacao'.")
    else:
        print("Aviso: Coluna 'Titulacao' não encontrada em 'df_tratado'.")

    if df_tratado is not None and 'Titulacao' in df_tratado.columns:
        print("Criando a coluna 'Nivel_Educacional' com base na coluna 'Titulacao'.")

        # Define o mapeamento de 'Titulacao' para uma nova coluna chamada 'Nivel_Educacional'
        titulacao_para_nivel = {
            'Não Informado': '(N/I) Não Informado',
            'Ensino Fundamental': '(0) Educação Básica',
            'Ensino Médio': '(0) Educação Básica',
            'Técnico': '(1) Educação Profissional',
            'Graduação': '(2) Educação Superior',
            'Aperfeiçoamento': '(3) Pós-graduação',
            'Especialização': '(3) Pós-graduação',
            'Mestrado': '(3) Pós-graduação',
            'Doutorado': '(3) Pós-graduação',
            'Aperfeicoamento': '(3) Pós-graduação',
            'Educação Básica': '(0) Educação Básica'
        }

        # Cria a nova coluna 'Nivel_Educacional' aplicando o mapeamento
        # Usa .map() para mapear os valores e .fillna() para garantir que valores não mapeados se tornem NaN ou outro indicador se necessário
        df_tratado['Nivel_Educacional'] = df_tratado['Titulacao'].map(titulacao_para_nivel).fillna('Outro/Não Mapeado') # Ajuste o fillna conforme a necessidade

    # Ajustes e Renomeação na coluna 'Campus_do_IFRN'
    if df_tratado is not None and 'Campus_do_IFRN' in df_tratado.columns:
        print("Realizando ajustes e renomeação na coluna 'Unidade_de_Lotacao'.")

        # Remove as expressões 'Campus ' e 'Campus Avançado '
        df_tratado['Campus_do_IFRN'] = df_tratado['Campus_do_IFRN'].str.replace('Campus Avançado ', '', regex=False)
        df_tratado['Campus_do_IFRN'] = df_tratado['Campus_do_IFRN'].str.replace('Campus ', '', regex=False)
        print("Expressões 'Campus ' e 'Campus Avançado ' removidas da coluna 'Campus_do_IFRN'.")

        # Renomeia parâmetros específicos em 'Campus_do_IFRN'
        unidade_replacements = {
            'Ceará-mirim': 'Ceará-Mirim',
            'Pau Dos Ferros': 'Pau dos Ferros',
            'Reitoria do Instituto Federal do Rio Grande do Norte': 'Reitoria'
        }
        df_tratado['Campus_do_IFRN'] = df_tratado['Campus_do_IFRN'].replace(unidade_replacements)
        print("Parâmetros específicos renomeados na coluna 'Campus_do_IFRN'.")

    # Exibir informações sobre as colunas e tipos de dados de 'df_tratado'
    print("\nInformações sobre as colunas e tipos de dados do DataFrame 'df_tratado':")
    df_tratado.info()

else:
    print("DataFrame 'df_filtrado' não encontrado. Execute as etapas anteriores.")

In [ ]:
# @title **Exibe novamente os valores únicos para as colunas indicadas em `df_tratado`**
if df_tratado is not None:
    columns_to_list = ['Classe', 'Jornada_de_Trabalho', 'Campus_do_IFRN', 'RSC', 'Titulacao', 'Vinculo_Carreira', 'Vinculo_Contrato',
                       'Vinculo_Professor', 'Nivel_Educacional']

    for col in columns_to_list:
        if col in df_tratado.columns:
            # Converte a coluna para o tipo 'string' antes de classificar valores exclusivos
            unique_values = sorted(df_tratado[col].astype(str).unique())
            print(f"Valores únicos em ordem alfabética na coluna '{col}':")
            for value in unique_values:
                print(f"- {value}")
            print("-" * 20) # Separador
        else:
            print(f"A coluna '{col}' não foi encontrada no DataFrame 'df_tratado'.")
else:
    print("DataFrame 'df_tratado' não encontrado. Execute as etapas anteriores.")

In [ ]:
    print("\nContagem de valores na nova coluna 'Nivel_Educacional':")
    display(df_tratado['Nivel_Educacional'].value_counts())

In [ ]:
# @title **Exibir informações sobre as colunas e tipos de dados de `df_tratado`**
if df_tratado is not None:
    print("Informações sobre as colunas e tipos de dados do DataFrame 'df_tratado':")
    df_tratado.info()
else:
    print("DataFrame 'df_tratado' não encontrado. Execute as etapas anteriores.")

In [ ]:
# @title **ETAPA 7: Exportação para o BigQuery**

# --- Configurações de Destino do BigQuery ---
import pandas_gbq

project_id = "pnp-data-extraction" # Substitua pelo ID do seu projeto
dataset_id = "pnp_dados_IFRN"      # Nome do conjunto de dados

# O nome da tabela no BigQuery será o mesmo nome da tabela da PNP
table_id = f"df_{table_name}"
destination_table = f"{dataset_id}.{table_id}"

# --- Execução da Exportação ---
if df_tratado is not None and not df_tratado.empty:
    print(f"Iniciando a exportação de {len(df_tratado):,} linhas para o BigQuery...")
    print(f"Destino: {project_id}.{destination_table}")

    # --- Renomear colunas para serem compatíveis com BigQuery ---
    # Substitui espaços e '/' por '_'
    df_tratado_bq = df_tratado.copy()
    df_tratado_bq.columns = (df_tratado_bq.columns
            .str.replace(' ', '_', regex=False)
            .str.replace('/', '_', regex=False)
            .str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
        )
    print("Nomes das colunas padronizados para o BigQuery.")

    # --- Definição do Esquema da Tabela ---
    # Defina o nome e o tipo de cada coluna que você quer controlar.
    # Tipos comuns: 'STRING', 'INTEGER', 'FLOAT', 'NUMERIC', 'BOOLEAN', 'TIMESTAMP', 'DATE'.
    table_schema = [
        {"name": "Classe", "type": "STRING"},
        {"name": "Cod_Unidade", "type": "STRING"},
        {"name": "Codigo_Municipio_com_DV", "type": "STRING"},
        {"name": "Codigo_da_Unidade_de_Ensino_-_SISTEC", "type": "STRING"},
        {"name": "Instituicao", "type": "STRING"},
        {"name": "Jornada_de_Trabalho", "type": "STRING"},
        {"name": "Municipio", "type": "STRING"},
        {"name": "Numero_de_registros", "type": "STRING"},
        {"name": "RSC", "type": "STRING"},
        {"name": "Regiao", "type": "STRING"},
        {"name": "Titulacao", "type": "STRING"},
        {"name": "Campus_do_IFRN", "type": "STRING"},
        {"name": "Vinculo_Carreira", "type": "STRING"},
        {"name": "Vinculo_Contrato", "type": "STRING"},
        {"name": "Vinculo_Professor", "type": "STRING"},
        {"name": "Nivel_Educacional", "type": "STRING"}
    ]
    # -----------------------------------------------------------------

    # Envia o DataFrame para o BigQuery com o esquema especificado
    try:
        # Usando a função recomendada pandas_gbq.to_gbq
        pandas_gbq.to_gbq(
            df_tratado_bq, # Exporta o DataFrame com colunas renomeadas
            destination_table=destination_table,
            project_id=project_id,
            if_exists='replace',  # Opções: 'fail', 'replace', 'append'
            progress_bar=True
        )
        print(f"\n✔ DataFrame exportado com sucesso para o BigQuery!")
        print(f"Link para a tabela: https://console.cloud.google.com/bigquery?project={project_id}&ws=!1m5!1m4!4m3!1s{project_id}!2s{dataset_id}!3s{table_id}")
    except Exception as e:
        print(f"❌ ERRO durante a exportação para o BigQuery: {e}")

elif df_tratado is not None and df_tratado.empty:
    print("AVISO: O DataFrame final está vazio. Nenhuma exportação foi realizada.")
else:
    print("❌ ERRO: O DataFrame a ser exportado não foi encontrado. A exportação foi cancelada.")